In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('data\gemstone.csv')

In [4]:
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [5]:
## let's drop the id column
df=df.drop(labels=['id'],axis=1)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
# Independent and Dependent feature
x=df.drop(labels=['price'],axis=1)
y=df['price']

In [7]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [8]:
# segrigate numerical and categorical value
categorical_cols=x.select_dtypes(include='object').columns
numerical_cols=x.select_dtypes(exclude='object').columns


In [9]:
# Define the custom ranking for each ordinal variable

cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
from sklearn.impute import SimpleImputer ## handling missing values
from sklearn.preprocessing import StandardScaler ## handling feature scaling
from sklearn.preprocessing import OrdinalEncoder ##ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

In [14]:
# categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),  
        ('OrdinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [15]:
## Train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.03,random_state=30)

In [16]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [17]:
x_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.195360,2.386873,0.401837,-0.121283,-0.144544,0.067195,-2.146403,0.297798,1.347351
1,-0.411571,-0.296021,-1.161107,-0.247488,-0.208053,-0.252187,0.872838,-1.550159,-0.649650
2,1.015422,0.166547,0.922818,1.104704,1.080272,1.097927,-0.133576,0.913784,0.016017
3,1.534329,-0.573562,1.443800,1.474303,1.452253,1.388274,-0.133576,0.913784,-1.315316
4,-0.973720,-0.203507,0.401837,-1.058803,-1.097178,-1.094193,-0.133576,-1.550159,0.016017


In [18]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [19]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [20]:
regression.coef_

array([ 6434.64156146,  -128.96112269,   -67.75425892, -1801.04395902,
        -388.90462069,   -93.66379143,    71.25405473,  -464.41060315,
         650.42310356])

In [21]:
regression.intercept_

3966.620818576415

In [24]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [27]:
## Train multiple models
# model evaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'RIDGE':Ridge(),
    'Elasticnet':ElasticNet()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make prediction

    y_pred=model.predict(x_test)
    mae,rmse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
Model Training performance
RMSE: 1031.4070277874346
MAE: 675.4950591797448
R2 score 93.65995253397321


Lasso
Model Training performance
RMSE: 1031.5981536487209
MAE: 676.7147669154795
R2 score 93.65760261920269


RIDGE
Model Training performance
RMSE: 1031.3976000707185
MAE: 675.514618125397
R2 score 93.66006843758194


Elasticnet
Model Training performance
RMSE: 1557.4843402609097
MAE: 1079.6753232674016
R2 score 85.54294896085551


